In [2]:
import gymnasium as gym
from gymnasium.wrappers import RescaleAction

base_env = gym.make("Hopper-v4")
base_env.action_space

/Users/yanhaonan/Documents/projects/study-rl/.venv/lib/python3.9/site-packages/gymnasium/envs/registration.py:519: DeprecationWarning: WARN: The environment Hopper-v4 is out of date. You should consider upgrading to version `v5`.
  logger.deprecation(


Box(-1.0, 1.0, (3,), float32)

In [3]:
wrapped_env = RescaleAction(base_env, min_action=0, max_action=1)
wrapped_env.action_space

Box(0.0, 1.0, (3,), float32)

In [4]:
wrapped_env

<RescaleAction<TimeLimit<OrderEnforcing<PassiveEnvChecker<HopperEnv<Hopper-v4>>>>>>

In [5]:
wrapped_env.env

<TimeLimit<OrderEnforcing<PassiveEnvChecker<HopperEnv<Hopper-v4>>>>>

In [6]:
wrapped_env.unwrapped

In [7]:
import numpy as np
import gymnasium as gym

env = gym.make("MountainCarContinuous-v0")
_ = env.reset(seed=123)
obs, *_= env.step(np.array([0.0, 1.0]))
obs

array([-4.6397772e-01, -4.4808415e-04], dtype=float32)

In [8]:
from gymnasium.wrappers import TransformAction

env = gym.make("MountainCarContinuous-v0")
env = TransformAction(env, lambda a: 0.5 * a + 0.1, env.action_space)
_ = env.reset(seed=123)
obs, *_= env.step(np.array([0.0, 1.0]))
obs

array([-4.6382770e-01, -2.9808417e-04], dtype=float32)

In [9]:
import gymnasium as gym
from gymnasium.wrappers import TimeAwareObservation
env = gym.make("CartPole-v1")
env = TimeAwareObservation(env)
env.observation_space

Box([-4.80000019        -inf -0.41887903        -inf  0.        ], [4.80000019e+00            inf 4.18879032e-01            inf
 5.00000000e+02], (5,), float64)

In [10]:
import numpy as np

import gymnasium as gym
from gymnasium import ActionWrapper, ObservationWrapper, RewardWrapper, Wrapper
from gymnasium.spaces import Box, Discrete

class RelativePosition(ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        self.observation_space = Box(shape=(2,), low=-np.inf, high=np.inf)

    def observation(self, obs):
        return obs["target"] - obs["agent"]

In [11]:
class DiscreteActions(ActionWrapper):
    def __init__(self, env, disc_to_cont):
        super().__init__(env)
        self.disc_to_cont = disc_to_cont
        self.action_space = Discrete(len(disc_to_cont))

    def action(self, act):
        return self.disc_to_cont[act]

env = gym.make("LunarLanderContinuous-v3")
print(env.action_space)  # Box(-1.0, 1.0, (2,), float32)

wrapped_env = DiscreteActions(
    env, [np.array([1, 0]), np.array([-1, 0]), np.array([0, 1]), np.array([0, -1])]
)
print(wrapped_env.action_space)  # Discrete(4)

Box(-1.0, 1.0, (2,), float32)
Discrete(4)


In [12]:
from typing import SupportsFloat


class ClipReward(RewardWrapper):
    def __init__(self, env, min_reward, max_reward):
        super().__init__(env)
        self.min_reward = min_reward
        self.max_reward = max_reward

    def reward(self, r: SupportsFloat) -> SupportsFloat:
        return np.clip(r, self.min_reward, self.max_reward)

In [13]:
class ReacherRewardWrapper(Wrapper):
    def __init__(self, env, reward_dist_weight, reward_ctrl_weight):
        super().__init__(env)
        self.reward_dist_weight = reward_dist_weight
        self.reward_ctrl_weight = reward_ctrl_weight

    def step(self, action):
        obs, _, terminated, truncated, info = self.env.step(action)
        reward = (
            self.reward_dist_weight * info["reward_dist"]
            + self.reward_ctrl_weight * info["reward_ctrl"]
        )
        return obs, reward, terminated, truncated, info